Circular Task Analysis

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import os
import re

# === PATHS ===
BASE_PATH = "data"  # folder where your CSV files are

# Professor data
DATA_FILE_PROF   = os.path.join(BASE_PATH, "001MoDe_R1.csv")
MARKER_FILE_PROF = os.path.join(BASE_PATH, "001MoDe_R1.marker.csv")

# Your own data (MouseReMoCo export)
MY_DATA_FILE     = os.path.join(BASE_PATH, "data.csv")
MY_MARKER_FILE   = os.path.join(BASE_PATH, "marker.csv")

# === CONSTANTS (from professor header) ===
SCREEN_WIDTH  = 1104
SCREEN_HEIGHT = 661
CORNER_X      = 302
CORNER_Y      = 80
CENTER_X      = 552
CENTER_Y      = 330
TASK_RADIUS   = 209.5
TASK_TOL      = 47.0   # taskTolerance (pixels)
PI            = math.pi